# Book-Oracle: Data Cleaning

- Clean threee datasets using functions in utils/data_cleaning & Save the merged dataset
- 26.11.2023
- Janina, Oliwia, Neha, Nina

## Import Libraries

In [1]:
import pandas as pd
import numpy as np

pd.options.display.float_format = "{:,.2f}".format

#Custom functions
from utils.data_cleaning import clean_ratings, clean_users, clean_books

RSEED = 42

import warnings
warnings.filterwarnings('ignore')

## Import Data

In [2]:
ratings_df = pd.read_csv('data/ratings.csv')
books_df = pd.read_csv('data/books.csv')
users_df = pd.read_csv('data/users.csv')

print("Ratings table has {} rows and {} columns".format(ratings_df.shape[0], ratings_df.shape[1]))
print("Books table has {} rows and {} columns".format(books_df.shape[0], books_df.shape[1]))
print("Users table has {} rows and {} columns".format(users_df.shape[0], users_df.shape[1]))

Ratings table has 1149780 rows and 3 columns
Books table has 271360 rows and 8 columns
Users table has 278858 rows and 3 columns


## Data cleaning

#### Clean Books Dataset

In [3]:
books_df, common_identifier_dict = clean_books(books_df)

print("Books table has {} rows and {} columns".format(books_df.shape[0], books_df.shape[1]))

Columns and their data types:
book_title             object
book_author            object
year_of_publication    object
publisher              object
image_url_m            object
common_identifier       int64
dtype: object
Books table has 239094 rows and 6 columns


#### Clean Ratings Dataset

CHANGE LAST PRINT!!

In [4]:
ratings_df = clean_ratings(ratings_df)

#assign common identifier and convert to int
ratings_df['common_identifier'] = ratings_df['isbn'].map(common_identifier_dict)

print("Ratings table has {} rows and {} columns".format(ratings_df.shape[0], ratings_df.shape[1]))

#check how many rows in ratings_df do not have a common identifier in books_df
print("")
print("There are {} rows in ratings_df that do not have a common identifier in books_df".format(ratings_df[~ratings_df['common_identifier'].isin(books_df['common_identifier'])].shape[0]))

ratings_df.sort_values(by="common_identifier", ascending=True).head(3)

Columns and their data types:
user_id         int64
isbn           object
book_rating     int64
dtype: object

Number of duplicated rows:0

Number of rows with missing values:0
Ratings table has 1149780 rows and 4 columns

There are 118646 rows in ratings_df that do not have a common identifier in books_df


,user_id,isbn,book_rating,common_identifier
9561,2,0195153448,0,1.00
1125456,269782,0801319536,7,1.00
188244,41385,0002005018,0,2.00


#### Clean Users Dataset

In [5]:
users_df = clean_users(users_df)

print("Users table has {} rows and {} columns".format(users_df.shape[0], users_df.shape[1]))
users_df.head(3)

Columns and their data types:
user_id     int64
age         int64
city       object
country    object
dtype: object
Users table has 275565 rows and 4 columns


,user_id,age,city,country
0,1,0,new york,usa
1,2,18,stockton,usa
2,3,0,moscow,russia


## Merge tables

For Content-based filtering (NLP) we will need to do a left merge to get ALL books regardless whether they have a rating. - then do LEFT merge.

Users - consider also RIGHT merge on users, to capture all users regarrdless whether there is metadata

In [6]:
#Merge all tables
df = books_df.merge(ratings_df, on='common_identifier', how='inner')
df = df.merge(users_df, on='user_id', how='inner')

#check for missing values
print("There are {} missing values in df".format(df.isnull().sum().sum()))

#check for duplicates
print("There are {} duplicates in df".format(df.duplicated().sum()))

df.head(3)

There are 0 missing values in df
There are 0 duplicates in df


,book_title,book_author,year_of_publication,publisher,image_url_m,common_identifier,user_id,isbn,book_rating,age,city,country
0,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,1,2,0195153448,0,18,stockton,usa
1,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,1,269782,0801319536,7,30,edmonton,canada
2,Pay It Forward: A Novel,Catherine Ryan Hyde,2000,Simon &amp; Schuster,http://images.amazon.com/images/P/0684862719.0...,2392,269782,0684862719,8,30,edmonton,canada


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1005487 entries, 0 to 1005486
Data columns (total 12 columns):
 #   Column               Non-Null Count    Dtype 
---  ------               --------------    ----- 
 0   book_title           1005487 non-null  object
 1   book_author          1005487 non-null  object
 2   year_of_publication  1005487 non-null  object
 3   publisher            1005487 non-null  object
 4   image_url_m          1005487 non-null  object
 5   common_identifier    1005487 non-null  int64 
 6   user_id              1005487 non-null  int64 
 7   isbn                 1005487 non-null  object
 8   book_rating          1005487 non-null  int64 
 9   age                  1005487 non-null  int64 
 10  city                 1005487 non-null  object
 11  country              1005487 non-null  object
dtypes: int64(4), object(8)
memory usage: 92.1+ MB


## Add descriptions to books  

In [8]:
#load descriptions
descriptions_df = pd.read_csv('data/books_w_descriptions.csv')

#merge descriptions to df based on common identifier    

df_full = df.merge(descriptions_df, on='common_identifier', how='left')

## Save cleaned data

In [9]:
df_full.to_csv('data/kaggle_full_df.csv', index=False)

In [10]:
df_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1005487 entries, 0 to 1005486
Data columns (total 13 columns):
 #   Column               Non-Null Count    Dtype 
---  ------               --------------    ----- 
 0   book_title           1005487 non-null  object
 1   book_author          1005487 non-null  object
 2   year_of_publication  1005487 non-null  object
 3   publisher            1005487 non-null  object
 4   image_url_m          1005487 non-null  object
 5   common_identifier    1005487 non-null  int64 
 6   user_id              1005487 non-null  int64 
 7   isbn                 1005487 non-null  object
 8   book_rating          1005487 non-null  int64 
 9   age                  1005487 non-null  int64 
 10  city                 1005487 non-null  object
 11  country              1005487 non-null  object
 12  description          641320 non-null   object
dtypes: int64(4), object(9)
memory usage: 99.7+ MB
